In [1]:
%matplotlib inline

import numpy as np
import scipy.linalg
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from numba import jit

In [2]:
@jit(nopython=True)
def build_pascal(lc,nc):
    cnkc = np.zeros((lc,nc),dtype=np.int64)
    for i in range(1,lc+1):
        cnkc[i-1,0] = 1
    for i in range(1,lc+1):
        for j in range(2,nc+1):
            cnkc[i-1,j-1] = 0
    for in1 in range(2,lc+1):
        cnkc[in1-1,1] = np.sum(cnkc[in1-2,0:2])
        if nc-1 > 1:
            for in2 in range(1,nc+1):
                cnkc[in1-1,in2-1] = np.sum(cnkc[in1-2,0:in2])
    return cnkc, cnkc[lc-1,nc-1]

In [3]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)
print(cnkc)
print(jmax)

[[  1   0   0   0   0   0   0]
 [  1   1   1   1   1   1   1]
 [  1   2   3   4   5   6   7]
 [  1   3   6  10  15  21  28]
 [  1   4  10  20  35  56  84]
 [  1   5  15  35  70 126 210]
 [  1   6  21  56 126 252 462]]
462


In [4]:
# Returns the position ind of the many body state bi
## original code, redundant if clauses
@jit(nopython=True)
def b2in_orig_redundant(bi,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
#    ind[0] = 0 ## start from 0
    for ind_i in range(1,lc-1):
        for ind_N in range(0,bi[ind_i-1]+1):
            if bi[ind_i-1] - ind_N > 0:
                suma = 0
                for k in range(1,ind_i):
                    suma += bi[k-1]
                if lc - ind_i > 0 and nc - ind_N - suma > 0:
                    ind[0] += cnkc[lc-ind_i-1,nc-ind_N-suma-1]
    return 0

In [5]:
# Returns the position ind of the many body state bi
@jit(nopython=True)
def b2in_orig(bi,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
#    ind[0] = 0 ## start from 0
    for ind_i in range(1,lc-1): # site loop
        for ind_N in range(0,bi[ind_i-1]): # atom loop
            suma = 0
            for k in range(1,ind_i):
                suma += bi[k-1]
            if nc - ind_N - suma > 0:
                ind[0] += cnkc[lc-ind_i-1,nc-ind_N-suma-1]
    return 0

In [6]:
# Returns the position ind of the many body state bi
@jit(nopython=True)
def b2in(bi,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
    num = nc
    for ind_site in range(1,lc-1): # site loop
        numb = bi[ind_site-1]
        ind[0] += np.sum(cnkc[lc-1-ind_site,num-numb:num])
        #print(ind_site,num-1,numb,cnkc[lc-1-ind_site,num-numb:num])
        num -= numb
    return 0

In [7]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

#bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind_orig = np.zeros(1,dtype=np.int64)

bi = np.array([2,1,1,0,2,0])
b2in_orig(bi,ind_orig,cnkc,lc,nc)
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind_orig[0],ind[0])

bi = np.array([1,0,3,0,2,0])
b2in_orig(bi,ind_orig,cnkc,lc,nc)
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind_orig[0],ind[0])

[2 1 1 0 2 0] 384 384
[1 0 3 0 2 0] 259 259


In [8]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

#bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

bi = np.array([0,0,0,0,0,6])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([1,1,1,1,1,1])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([6,0,0,0,0,0])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([1,0,3,0,2,0])
#bi = np.array([0,2,0,3,0,1])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([2,1,1,0,2,0])
#bi = np.array([0,2,0,1,1,2])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

[0 0 0 0 0 6] 1
[1 1 1 1 1 1] 287
[6 0 0 0 0 0] 462
[1 0 3 0 2 0] 259
[2 1 1 0 2 0] 384


In [9]:
# Returns the position ind of the many body state bi_short
@jit(nopython=True)
def bshort2in(bis,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
    num = 2
    for ind_atom in range(1,nc): # atom loop
        ind_site = bis[ind_atom-1]
        ind[0] += cnkc[ind_site-1,num-1]
        #print(ind_atom,ind_site,num,cnkc[ind_site-1,num-1],ind[0])
        num += 1
    return 0

In [10]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

#bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

bis = np.array([2,2,4,5,6,6])
bshort2in(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

[2 2 4 5 6 6] 384


In [11]:
# Returns the many body state bi at position ind
@jit(nopython=True)
def in2b(bi,ind,cnkc,lc,nc): # index to basis
    ind_i = ind[0] - 1 ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bi[:] = 0
    ind_L = lc - 1
    ind_N = nc
    while ind_N > 1: # atom loop
        if ind_i >= cnkc[ind_L-1,ind_N-1]: # condition for site
            ind_i -= cnkc[ind_L-1,ind_N-1]
            bi[lc-ind_L-1] += 1
            ind_N -= 1
        else:
            ind_L -= 1
    return 0

In [12]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind[0] = 384
in2b(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

[2 1 1 0 2 0] 384


In [13]:
# Returns the many body state bi_short at position ind
@jit(nopython=True)
def in2bshort(bis,ind,cnkc,lc,nc): # index to basis short
    ind_i = ind[0] - 1 ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bis[:] = 0
    ind_L = lc - 1
    ind_N = nc
    while ind_N > 1: # atom loop
        if ind_i >= cnkc[ind_L-1,ind_N-1]: # condition for site
            ind_i -= cnkc[ind_L-1,ind_N-1]
            bis[ind_N-2] = ind_L
            ind_N -= 1
        else:
            ind_L -= 1
    return 0

In [14]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind[0] = 384
in2bshort(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

[2 2 4 5 6 6] 384


In [15]:
@jit(nopython=True)
def binary_search(s,list_s,ls,le):
    bmin = ls; bmax = le
    while True:
        b = bmin + (bmax-bmin)//2
        if s < list_s[b-1]:
            bmax = b - 1
        elif list_s[b-1] < s:
            bmin = b + 1
        else:
            bmin = b
            return b, bmin
        if bmin > bmax:
            b = -1
            return b, bmin
    return b, bmin

In [16]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

print(cnkc[:,nc-1])
print()

for i in cnkc[:,nc-1]-2:
    b, bmin = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin)
print()

for i in cnkc[:,nc-1]-1:
    b, bmin = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin)
print()

for i in cnkc[:,nc-1]:
    b, bmin = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin)
print()

for i in cnkc[:,nc-1]+1:
    b, bmin = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin)
print()

for i in cnkc[:,nc-1]+2:
    b, bmin = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin)
print()

[  0   1   7  28  84 210 462]

-2 -1 1
-1 -1 1
5 -1 3
26 -1 4
82 -1 5
208 -1 6
460 -1 7

-1 -1 1
0 1 1
6 -1 3
27 -1 4
83 -1 5
209 -1 6
461 -1 7

0 1 1
1 2 2
7 3 3
28 4 4
84 5 5
210 6 6
462 7 7

1 2 2
2 -1 3
8 -1 4
29 -1 5
85 -1 6
211 -1 7
463 -1 8

2 -1 3
3 -1 3
9 -1 4
30 -1 5
86 -1 6
212 -1 7
464 -1 8



In [17]:
# Returns the many body state bi_short at position ind
# using binary search
@jit(nopython=True)
def in2bshort_bs(bis,ind,cnkc,lc,nc): # index to basis short
    ind_i = ind[0] ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bis[:] = 0
    ind_site = lc
    for ind_atom in range(nc,1,-1): # atom loop
        icnkc, icnkcmin = binary_search(ind_i,cnkc[:ind_site,ind_atom-1],1,ind_site)
        #print(ind_atom,ind_i,icnkc,icnkcmin,cnkc[:ind_site,ind_atom-1],cnkc[icnkcmin-2,ind_atom-1])
        ind_i -= cnkc[icnkcmin-2,ind_atom-1]
        bis[ind_atom-2] = icnkcmin-1
        ind_site = icnkcmin
    return 0

In [18]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

ind[0] = 384
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

ind[0] = 259
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

ind[0] = 1
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

ind[0] = jmax
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

[2 2 4 5 6 6] 384

[2 2 4 4 4 6] 259

[1 1 1 1 1 1] 1

[6 6 6 6 6 6] 462



In [19]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bi = np.zeros(lc-1,dtype=np.int64)
bis = np.zeros(nc-1,dtype=np.int64)
bis_bs = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind_bi = np.zeros(1,dtype=np.int64)
ind_bis = np.zeros(1,dtype=np.int64)
ind_bis_bs = np.zeros(1,dtype=np.int64)

#for i in range(1,jmax+1):
for i in list(range(1,9))+[259]+[384]+list(range(jmax+1-8,jmax+1)):
    ind[0] = i
    in2b(bi,ind,cnkc,lc,nc)
    in2bshort(bis,ind,cnkc,lc,nc)
    in2bshort_bs(bis_bs,ind,cnkc,lc,nc)
    b2in(bi,ind_bi,cnkc,lc,nc)
    bshort2in(bis,ind_bis,cnkc,lc,nc)
    bshort2in(bis_bs,ind_bis_bs,cnkc,lc,nc)
    print(ind[0],bi,ind_bi[0]," ",ind[0],bis,ind_bis[0]," ",ind[0],bis_bs,ind_bis_bs[0])

1 [0 0 0 0 0 6] 1   1 [1 1 1 1 1 1] 1   1 [1 1 1 1 1 1] 1
2 [0 0 0 0 1 5] 2   2 [1 1 1 1 1 2] 2   2 [1 1 1 1 1 2] 2
3 [0 0 0 0 2 4] 3   3 [1 1 1 1 2 2] 3   3 [1 1 1 1 2 2] 3
4 [0 0 0 0 3 3] 4   4 [1 1 1 2 2 2] 4   4 [1 1 1 2 2 2] 4
5 [0 0 0 0 4 2] 5   5 [1 1 2 2 2 2] 5   5 [1 1 2 2 2 2] 5
6 [0 0 0 0 5 1] 6   6 [1 2 2 2 2 2] 6   6 [1 2 2 2 2 2] 6
7 [0 0 0 0 6 0] 7   7 [2 2 2 2 2 2] 7   7 [2 2 2 2 2 2] 7
8 [0 0 0 1 0 5] 8   8 [1 1 1 1 1 3] 8   8 [1 1 1 1 1 3] 8
259 [1 0 3 0 2 0] 259   259 [2 2 4 4 4 6] 259   259 [2 2 4 4 4 6] 259
384 [2 1 1 0 2 0] 384   384 [2 2 4 5 6 6] 384   384 [2 2 4 5 6 6] 384
455 [4 1 1 0 0 0] 455   455 [4 5 6 6 6 6] 455   455 [4 5 6 6 6 6] 455
456 [4 2 0 0 0 0] 456   456 [5 5 6 6 6 6] 456   456 [5 5 6 6 6 6] 456
457 [5 0 0 0 0 1] 457   457 [1 6 6 6 6 6] 457   457 [1 6 6 6 6 6] 457
458 [5 0 0 0 1 0] 458   458 [2 6 6 6 6 6] 458   458 [2 6 6 6 6 6] 458
459 [5 0 0 1 0 0] 459   459 [3 6 6 6 6 6] 459   459 [3 6 6 6 6 6] 459
460 [5 0 1 0 0 0] 460   460 [4 6 6 6 6 6] 460 

In [20]:
#lc = 33 # number of sites + 1
lc = 25 # number of sites + 1
nc = 4 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bi = np.zeros(lc-1,dtype=np.int64)
bis = np.zeros(nc-1,dtype=np.int64)
bis_bs = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind_bi = np.zeros(1,dtype=np.int64)
ind_bis = np.zeros(1,dtype=np.int64)
ind_bis_bs = np.zeros(1,dtype=np.int64)

#for i in range(1,jmax+1):
for i in list(range(1,9))+list(range(jmax+1-8,jmax+1)):
    ind[0] = i
    in2b(bi,ind,cnkc,lc,nc)
    in2bshort(bis,ind,cnkc,lc,nc)
    in2bshort_bs(bis_bs,ind,cnkc,lc,nc)
    b2in(bi,ind_bi,cnkc,lc,nc)
    bshort2in(bis,ind_bis,cnkc,lc,nc)
    bshort2in(bis_bs,ind_bis_bs,cnkc,lc,nc)
    print(ind[0],bi,ind_bi[0]," ",ind[0],bis,ind_bis[0]," ",ind[0],bis_bs,ind_bis_bs[0])

1 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3] 1   1 [1 1 1] 1   1 [1 1 1] 1
2 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2] 2   2 [1 1 2] 2   2 [1 1 2] 2
3 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1] 3   3 [1 2 2] 3   3 [1 2 2] 3
4 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0] 4   4 [2 2 2] 4   4 [2 2 2] 4
5 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2] 5   5 [1 1 3] 5   5 [1 1 3] 5
6 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1] 6   6 [1 2 3] 6   6 [1 2 3] 6
7 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0] 7   7 [2 2 3] 7   7 [2 2 3] 7
8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1] 8   8 [1 3 3] 8   8 [1 3 3] 8
2593 [2 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 2593   2593 [17 24 24] 2593   2593 [17 24 24] 2593
2594 [2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 2594   2594 [18 24 24] 2594   2594 [18 24 24] 2594
2595 [2 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 2595   2595 [19 24 24] 2595   2595 [19 24 24] 2595
2596 [2 0 0 0 1 0 0 0 0 0 

In [21]:
@jit(nopython=True)
def mkncfg(bis,tmp_n_site,tmp_n_atom):
    tmp_n_site[:] = 0
    tmp_n_atom[:] = 0
    cnt = 0
    tmp_n_site[cnt] = bis[cnt]
    tmp_n_atom[cnt] = 1
    for i in range(1,len(bis)):
        if bis[i] > bis[i-1]:
            cnt += 1
            tmp_n_site[cnt] = bis[i]
            tmp_n_atom[cnt] = 1
        else:
            tmp_n_atom[cnt] += 1
    return 0

In [22]:
@jit(nopython=True)
def mknidx(bis,tmp_n_site,tmp_n_atom):
    bis[:] = 0
    cnt = 0
    for i,n in enumerate(tmp_n_atom):
        for _ in range(n):
            bis[cnt] = tmp_n_site[i]
            #print(i,cnt,n,tmp_n_site[i])
            cnt += 1
    return 0

In [23]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
tmp_n_site = np.zeros(nc-1,dtype=np.int64)
tmp_n_atom = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)

ind[0] = 384
in2bshort_bs(bis,ind,cnkc,lc,nc)
mkncfg(bis,tmp_n_site,tmp_n_atom)
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(bis,ind[0],tmp_n_site,tmp_n_atom,bis2)
print()

ind[0] = 259
in2bshort_bs(bis,ind,cnkc,lc,nc)
mkncfg(bis,tmp_n_site,tmp_n_atom)
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(bis,ind[0],tmp_n_site,tmp_n_atom,bis2)
print()

ind[0] = 1
in2bshort_bs(bis,ind,cnkc,lc,nc)
mkncfg(bis,tmp_n_site,tmp_n_atom)
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(bis,ind[0],tmp_n_site,tmp_n_atom,bis2)
print()

ind[0] = jmax
in2bshort_bs(bis,ind,cnkc,lc,nc)
mkncfg(bis,tmp_n_site,tmp_n_atom)
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(bis,ind[0],tmp_n_site,tmp_n_atom,bis2)
print()

[2 2 4 5 6 6] 384 [2 4 5 6 0 0] [2 1 1 2 0 0] [2 2 4 5 6 6]

[2 2 4 4 4 6] 259 [2 4 6 0 0 0] [2 3 1 0 0 0] [2 2 4 4 4 6]

[1 1 1 1 1 1] 1 [1 0 0 0 0 0] [6 0 0 0 0 0] [1 1 1 1 1 1]

[6 6 6 6 6 6] 462 [6 0 0 0 0 0] [6 0 0 0 0 0] [6 6 6 6 6 6]



In [24]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
tmp_n_site = np.zeros(nc-1,dtype=np.int64)
tmp_n_atom = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)

tmp_n_site = np.array([2,4,5,6,0,0])
tmp_n_atom = np.array([2,1,1,2,0,0])
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(tmp_n_site,tmp_n_atom,bis2)
print()

tmp_n_site = np.array([2,3,4,5,6,0])
tmp_n_atom = np.array([2,0,1,1,2,0])
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(tmp_n_site,tmp_n_atom,bis2)
print()

tmp_n_site = np.array([4,5,2,6,0,0])
tmp_n_atom = np.array([1,1,2,2,0,0])
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(tmp_n_site,tmp_n_atom,bis2)
bis2.sort()
print(tmp_n_site,tmp_n_atom,bis2)
print()

tmp_n_site = np.array([6,5,4,3,2,1])
tmp_n_atom = np.array([2,1,1,0,2,0])
mknidx(bis2,tmp_n_site,tmp_n_atom)
print(tmp_n_site,tmp_n_atom,bis2)
bis2.sort()
print(tmp_n_site,tmp_n_atom,bis2)
print()

[2 4 5 6 0 0] [2 1 1 2 0 0] [2 2 4 5 6 6]

[2 3 4 5 6 0] [2 0 1 1 2 0] [2 2 4 5 6 6]

[4 5 2 6 0 0] [1 1 2 2 0 0] [4 5 2 2 6 6]
[4 5 2 6 0 0] [1 1 2 2 0 0] [2 2 4 5 6 6]

[6 5 4 3 2 1] [2 1 1 0 2 0] [6 6 5 4 2 2]
[6 5 4 3 2 1] [2 1 1 0 2 0] [2 2 4 5 6 6]

